In [1]:
from argparse import ArgumentParser
import os
import shutil
from tqdm import tqdm
import numpy as np
import torch
from utils import listdir, image_to_tensor, word2sense, p2r, cv_regression_w2s
import matplotlib.pyplot as plt
from collections import OrderedDict 
import warnings; warnings.simplefilter('ignore')
import string

scaffold = np.load(os.path.join('saved_weights', 'EVC_fc6_subj001') + '.npy')
permutation = np.random.choice(scaffold.shape[1], scaffold.shape[1], replace=False)
print(permutation)
idx = np.empty_like(permutation)
idx[permutation] = np.arange(len(permutation))
weight = scaffold[:, idx]

# fig = plt.figure(figsize=(20, 25))
# plt.imshow(scaffold)
# plt.show()
# fig = plt.figure(figsize=(20, 25))
# plt.imshow(weight)
# plt.show()

[2556 2858 2705 ... 1746 3351  887]


In [2]:
# Setup w2s
w2s_file = "ThingsWrd2Sns.txt"
_, w2s_embeddings = word2sense(w2s_file)
nDim = 2250

# Setup wordvec, sensevec
embedding_file = "sensevec_augmented_with_wordvec.csv"
f = open(embedding_file, 'r', encoding='utf-8')
senseveclines = f.readlines()
f.close()

embedding_file = "wordvec.csv"
f = open(embedding_file, 'r', encoding='utf-8')
wordveclines = f.readlines()
f.close()

# Cat's new word2vec (312 categories)
embedding_file = "ThingsWrd2Vec_subset.txt"
f = open(embedding_file, 'r', encoding='utf-8')
wordvec2lines = f.readlines()
f.close()

wordvec = {}
sensevec = {}
wordvec2 = {}

things = sorted(os.listdir('predicted_EVC_conv5'))

for i, thing in enumerate(things):
    if thing in w2s_embeddings:
        wordvec[thing] = wordveclines[i].split(',')
        wordvec[thing][-1] = wordvec[thing][-1].replace('\n', '')
        sensevec[thing] = senseveclines[i].split(',')
        sensevec[thing][-1] = sensevec[thing][-1].replace('\n', '')

# Wordvec without nan
wordvec_nonan = wordvec
nankeys = []
for wv in wordvec:
    ar = np.array(wordvec[wv])
    ar = ar.astype(np.float)
    if np.isnan(ar).any():
        nankeys.append(wv)

print(len(nankeys))  
 
for n in nankeys:
    del wordvec_nonan[n]

for wv in wordvec_nonan:
    wordvec_nonan[wv] = np.array(wordvec_nonan[wv]).astype(np.float)
    
print(len(wordvec_nonan.keys()))

# Wordvec filtered by CM
for i, wv2 in enumerate(wordvec2lines):
    if i == 0:
        continue
    word = (wordvec2lines[i].split(','))[0]
    if word in w2s_embeddings:
        wordvec2[word] = wordvec2lines[i].split(',')[1:]
        wordvec2[word][-1] = wordvec2[word][-1].replace('\n', '')

for wv in wordvec2:
    wordvec2[wv] = np.array(wordvec2[wv]).astype(np.float)
    

# Sensevec
for i, thing in enumerate(things):
    if thing in w2s_embeddings:
        sensevec[thing] = senseveclines[i].split(',')
        sensevec[thing][-1] = sensevec[thing][-1].replace('\n', '')
            
sensevec_nonan = sensevec
nankeys = []
for sv in sensevec:
    ar = np.array(sensevec[sv])
    ar = ar.astype(np.float)
    if np.isnan(ar).any():
        nankeys.append(sv)

print(nankeys)  
 
for n in nankeys:
    del sensevec_nonan[n]

for sv in sensevec_nonan:
    sensevec_nonan[sv] = np.array(sensevec_nonan[sv]).astype(np.float)
    
print(len(sensevec_nonan.keys()))

# Sensevec filtered to 312 categories
sensevec_filtered = {}

for i, wv2 in enumerate(wordvec2lines):
    if i == 0:
        continue
    word =  (wordvec2lines[i].split(','))[0]
    if word in sensevec.keys():
        sensevec_filtered[word] = sensevec[word]

# Wordvec (osf) filtered to 312 categories
wordvec_filtered = {}

for i, wv2 in enumerate(wordvec2lines):
    if i == 0:
        continue
    word =  (wordvec2lines[i].split(','))[0]
    if word in wordvec.keys():
        wordvec_filtered[word] = wordvec[word]
    
# dimensionality score
r_threshold = p2r(0.05/nDim, 1470)
r_threshold_wordvec = p2r(0.05/300, len(wordvec2lines)-1)

nPC = 7;

110
1360
['handcuff', 'paperweight', 'peppermint', 'pug', 'slime', 'teacup']
1464


In [ ]:
# Stack data
rois = ['LOC', 'PPA', 'EVC'] 
layers = ['conv5', 'fc6'] #'conv1', 'conv5', 'fc6', 'conv1_r', 'conv5_r', 'fc6_r'
subjects = ['subj001/', 'subj002/', 'subj003/', 'subj004/'] #subj001/ , 'subj002/', 'subj003/', 'subj004/'
random_subjects = ['subj001_random/', 'subj002_random/', 'subj003_random/', 'subj004_random/']

weighttype = []
predictpath = {
    "real": "predicted1023/predicted/",
    "random": "predictedww/"
}

# Initialize r values
#roi_r_set = {}
#random_r_set = {}

# Permutation repeated many times
perm_r_set = []
for i in range(0, 100):
    weighttype.append("perm" + str(i))
    predictpath["perm" + str(i)] = "predicted1023/predictedperm" + str(i) + "/"
    perm_r_set.append({})

print(weighttype)
print(predictpath)


for weight in weighttype:
    if weight == "random":
        subjects = random_subjects
    else:
        subjects = ['subj001/', 'subj002/', 'subj003/', 'subj004/']
    for roi in rois:
        roi_r = []
        for layer in layers:
            sub_voxel_regressor = {}

            for subj in subjects:
                path = predictpath[weight]
                fmri_path = path + subj + roi + "_" + layer
                fmri_path = os.path.join(os.getcwd(), fmri_path) 

                conditions = sorted(listdir(fmri_path))
                condition_voxels = {}
                for condition in conditions:
                    if condition.split('/')[-1] in w2s_embeddings.keys():
                        file_name = listdir((os.path.join(fmri_path, condition)))[0]
                        file_path = os.path.join(fmri_path, condition, file_name)
                        condition_voxels[condition] = np.load(file_path) 
                voxel_regressor = np.stack([condition_voxel for condition, condition_voxel in OrderedDict(condition_voxels).items()])
                sub_voxel_regressor[subj] = voxel_regressor

                #print(sub_voxel_regressor[subj].shape)
            all_voxel_regressor = np.hstack([sub_voxel_regressor[s] for s in subjects])
            #print(all_voxel_regressor.shape)

            for pc in tqdm(range(10, 80, 10), total=7, position=0, leave=True):
                _, voxel_mean_r = cv_regression_w2s(all_voxel_regressor, w2s_embeddings, fit=None, k=9, l2=0.0, pc=pc)
                roi_r.append(voxel_mean_r)

            #if subj == 'random/':
            #random_r_set[roi] = roi_r
            #else:
            if weight == "real":
                roi_r_set[roi] = roi_r
            elif weight == "random":
                random_r_set[roi] = roi_r
            elif "perm" in weight:
                i = int(weight.strip(string.ascii_letters))
                print(i)
                perm_r_set[i][roi] = roi_r

['perm0', 'perm1', 'perm2', 'perm3', 'perm4', 'perm5', 'perm6', 'perm7', 'perm8', 'perm9', 'perm10', 'perm11', 'perm12', 'perm13', 'perm14', 'perm15', 'perm16', 'perm17', 'perm18', 'perm19', 'perm20', 'perm21', 'perm22', 'perm23', 'perm24', 'perm25', 'perm26', 'perm27', 'perm28', 'perm29', 'perm30', 'perm31', 'perm32', 'perm33', 'perm34', 'perm35', 'perm36', 'perm37', 'perm38', 'perm39', 'perm40', 'perm41', 'perm42', 'perm43', 'perm44', 'perm45', 'perm46', 'perm47', 'perm48', 'perm49', 'perm50', 'perm51', 'perm52', 'perm53', 'perm54', 'perm55', 'perm56', 'perm57', 'perm58', 'perm59', 'perm60', 'perm61', 'perm62', 'perm63', 'perm64', 'perm65', 'perm66', 'perm67', 'perm68', 'perm69', 'perm70', 'perm71', 'perm72', 'perm73', 'perm74', 'perm75', 'perm76', 'perm77', 'perm78', 'perm79', 'perm80', 'perm81', 'perm82', 'perm83', 'perm84', 'perm85', 'perm86', 'perm87', 'perm88', 'perm89', 'perm90', 'perm91', 'perm92', 'perm93', 'perm94', 'perm95', 'perm96', 'perm97', 'perm98', 'perm99']
{'real': 

100%|██████████| 7/7 [00:18<00:00,  2.58s/it]


0


100%|██████████| 7/7 [00:17<00:00,  2.53s/it]


0


...

100%|██████████| 7/7 [00:20<00:00,  2.87s/it]


60


100%|██████████| 7/7 [00:18<00:00,  2.66s/it]


60


 57%|█████▋    | 4/7 [00:11<00:08,  2.73s/it]